<a href="https://colab.research.google.com/github/Hsuan7/homework/blob/main/%E9%87%91%E8%9E%8D%E5%A4%A7%E6%95%B8%E6%93%9Aw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

從證券交易所爬取股價資料並進行可視化

In [22]:
import requests
import pandas as pd
from datetime import datetime
from urllib.parse import urlencode

# https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY?date=20240923&stockNo=2330&response=json&_=1727096084039
# 定義函數來生成動態的URL
def get_twse_url(stock_no, date=None):
    if date is None:
        # 自動提取當前日期，格式為 'YYYYMMDD'
        date = datetime.now().strftime('%Y%m%d')

    base_url = 'https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY'
    params = {
        'date': date,
        'stockNo': stock_no,
        'response': 'json'
    }
    return f'{base_url}?{urlencode(params)}'

# 設定查詢股票代號
stock_no = '2330'  # 台積電股票代號

# 生成動態URL，自動提取當前日期
url = get_twse_url(stock_no)

# 發送GET請求
res = requests.get(url)

# 將回應轉換為JSON格式
json_data = res.json()

# 引入pandas庫
import pandas as pd

# 將JSON數據轉換為DataFrame
df = pd.DataFrame(data=json_data['data'],columns=json_data['fields'])

# 顯示DataFrame
df



,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,113/09/02,"19,272,593","18,270,058,260",950.00,955.00,943.00,948.00,+4.00,"31,642"
1,113/09/03,"23,205,623","21,908,471,541",948.00,952.00,939.00,940.00,-8.00,"47,139"
2,113/09/04,"93,169,835","83,424,133,824",894.00,905.00,888.00,889.00,-51.00,"393,210"
3,113/09/05,"34,147,890","30,998,595,394",907.00,915.00,900.00,902.00,+13.00,"63,902"
4,113/09/06,"28,248,063","25,786,016,936",909.00,918.00,903.00,918.00,+16.00,"38,415"
5,113/09/09,"38,448,946","34,456,838,126",892.00,900.00,891.00,899.00,-19.00,"102,365"
6,113/09/10,"34,312,646","31,067,424,234",907.00,911.00,901.00,904.00,+5.00,"45,141"
7,113/09/11,"19,513,256","17,621,000,956",906.00,906.00,900.00,901.00,-3.00,"30,789"
8,113/09/12,"43,749,260","40,999,951,508",936.00,944.00,928.00,940.00,X0.00,"80,643"
9,113/09/13,"28,307,441","26,795,862,243",955.00,955.00,939.00,947.00,+7.00,"39,071"



從Yahoo奇摩爬取股價資料並進行可視化


In [23]:
!pip install requests beautifulsoup4


In [24]:
from bs4 import BeautifulSoup
def yahoo_stock(stock_id):
    # 根據傳入的股票代號生成Yahoo奇摩股票頁面的URL
    url = f'https://tw.stock.yahoo.com/quote/{stock_id}.TW'

    # 發送GET請求
    response = requests.get(url)

    # 確保請求成功，獲取HTML文本
    html = response.content

    # 使用BeautifulSoup解析獲取的HTML內容
    soup = BeautifulSoup(html, 'html.parser')

     # 定位到特定的section，並查找其中的time元素以獲取資料時間
    time_element = soup.find('section', {'id': 'qsp-overview-realtime-info'}).find('time')

    # 獲取同一section下的ul標籤，並提取所有的li標籤
    table_soups = soup.find('section', {'id': 'qsp-overview-realtime-info'}).find('ul').find_all('li')

    # 初始化欄位名稱列表
    fields = []
    # 初始化存儲數據的列表
    datas = []

    # 遍歷每個li標籤以提取其內容
    for table_soup in table_soups:

        # 查找每個li內的所有span標籤
        table_datas = table_soup.find_all('span')

        for num, table_data in enumerate(table_datas):
            # 如果文本為空則跳過
            if table_data.text == '':
                continue

            # 將第一個span的文本作為欄位名稱，其他作為數據
            if num == 0:
                fields.append(table_data.text)
            else:
                datas.append(table_data.text)

    # 建立 DataFrame
    df = pd.DataFrame([datas], columns=fields)

    # 增加日期和股號欄位
    df.insert(0,'日期',time_element['datatime'])
    df.insert(1,'股號',stock_no)

    # 顯示DataFrame
    return df

# 設定查詢股票代號
stock_no = '2330'  # 台積電股票代號

# 呼叫函數
yahoo_stock(stock_no)

,日期,股號,成交,開盤,最高,最低,均價,成交金額(億),昨收,漲跌幅,漲跌,總量,昨量,振幅
0,2024/09/23 14:30,2330,977,971,977,971,975,140.96,973,0.41%,4.00,"14,456","60,066",0.62%
